In [ ]:
import argparse
import os
import numpy as np
import math


import torchvision.transforms as transforms
from torchvision.utils import save_image
#from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import math
import warnings
import numpy as np
from scipy import linalg
import torch
import torch.nn.functional as F
import torchvision.models as models

import time
from google.colab import drive
drive.mount('/content/drive/')


from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset  # For custom datasets

from tqdm import tqdm
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

In order to log the values of discriminator and generator loss using the weights and biases, a user account has to be created under **weights and biases** api.

In [ ]:
!pip install wandb -qU
import wandb
wandb.login()



In [ ]:
#Initialising weights and biases API to track generation ans=d discrimination losses

wandb.init(project="DCGAN_Wano_1000epochs",name='Loss_curve',config={
      "learning_rate": 0.0002,
      "architecture": "DCGAN",
      "dataset": "cifar",
      "epochs":n_epochs,
      }) 

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(0.2),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(0.2),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img





In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity




In [ ]:
os.makedirs("images", exist_ok=True)

n_epochs=1000  # number of epochs of training
batch_size=64 # size of the batches
lr=0.0002 # adam: learning rate
b1=0.5  # adam: decay of first order momentum of gradient
b2=0.999  # adam: decay of second order momentum of gradient
n_cpu=8   #  number of cpu threads to use during batch generation
latent_dim=100 #  dimensionality of the latent space
img_size=32   # size of each image dimension
channels=3    #number of image channels
sample_interval=400 # interval between image sampling
cuda = True if torch.cuda.is_available() else False

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)
print(generator)
print(discriminator)

# Configure data loader
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.SVHN(
        "../../data/mnist",split='train',
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

'''
# Uncomment the following to train GANs using CIFAR10 dataset
dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        "../../data/mnist",train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

'''

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
class FID():
    
    def __init__(self, cache_dir='./Cache',  device='cpu',transform_input=True):
        os.environ["TORCH_HOME"] = "./Cache"
        self.device=device
        self.transform_input = transform_input
        self.InceptionV3 = models.inception_v3(pretrained=True, transform_input=False, aux_logits=True).to(device=self.device)
        self.InceptionV3.eval()
    
    def build_maps(self, x):
        # Resize to Fit InceptionV3
        if list(x.shape[-2:]) != [299,299]:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                x = F.interpolate(x, size=[299,299], mode='bilinear')
        # Transform Input to InceptionV3 Standards
        if self.transform_input:
            a = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            b = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            c = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((a,b,c), 1)
        # Run Through Partial InceptionV3 Model
        with torch.no_grad():
            # N x 3 x 299 x 299
            x = self.InceptionV3.Conv2d_1a_3x3(x)
            # N x 32 x 149 x 149
            x = self.InceptionV3.Conv2d_2a_3x3(x)
            # N x 32 x 147 x 147
            x = self.InceptionV3.Conv2d_2b_3x3(x)
            # N x 64 x 147 x 147
            x = F.max_pool2d(x, kernel_size=3, stride=2)
            # N x 64 x 73 x 73
            x = self.InceptionV3.Conv2d_3b_1x1(x)
            # N x 80 x 73 x 73
            x = self.InceptionV3.Conv2d_4a_3x3(x)
            # N x 192 x 71 x 71
            x = F.max_pool2d(x, kernel_size=3, stride=2)
            # N x 192 x 35 x 35
            x = self.InceptionV3.Mixed_5b(x)
            # N x 256 x 35 x 35
            x = self.InceptionV3.Mixed_5c(x)
            # N x 288 x 35 x 35
            x = self.InceptionV3.Mixed_5d(x)
            # N x 288 x 35 x 35
            x = self.InceptionV3.Mixed_6a(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6b(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6c(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6d(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_6e(x)
            # N x 768 x 17 x 17
            x = self.InceptionV3.Mixed_7a(x)
            # N x 1280 x 8 x 8
            x = self.InceptionV3.Mixed_7b(x)
            # N x 2048 x 8 x 8
            x = self.InceptionV3.Mixed_7c(x)
            # N x 2048 x 8 x 8
            # Adaptive average pooling
            x = F.adaptive_avg_pool2d(x, (1, 1))
            # N x 2048 x 1 x 1
            return x
  

    def compute_fid(self, real_img, generated_img, batch_size=64):
        # Ensure Set Sizes are the Same
        assert(real_img.shape[0] == generated_img.shape[0])
        # Build Random Sampling Orders
        real_img = real_img[np.random.permutation(real_img.shape[0])]
        generated_imag = generated_img[np.random.permutation(generated_img.shape[0])]
        # Lists of Maps per Batch
        real_maps = []
        gen_maps = []
        # Build Maps
        for s in range(int(math.ceil(real_img.shape[0]/batch_size))):
            sidx = np.arange(batch_size*s, min(batch_size*(s+1), real_img.shape[0]))
            real_maps.append(self.build_maps(real_img[sidx].to(device=self.device)).detach().to(device='cpu'))
            gen_maps.append(self.build_maps(generated_imag[sidx].to(device=self.device)).detach().to(device='cpu'))

        # Concatenate Maps
        real_maps = np.squeeze(torch.cat(real_maps).numpy())
        gen_maps = np.squeeze(torch.cat(gen_maps).numpy())
        # Calculate FID
        # Activation Statistics
        mu_g = np.mean(gen_maps, axis=0)
        mu_x = np.mean(real_maps, axis=0)
        sigma_g = np.cov(gen_maps, rowvar=False)
        sigma_x = np.cov(real_maps, rowvar=False)
        # Sum of Squared Differences
        ssd = np.sum((mu_g - mu_x)**2)
        # Square Root of Product of Covariances
        covmean = linalg.sqrtm(sigma_g.dot(sigma_x), disp=False)[0]
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        # Final FID Computation
        return ssd + np.trace(sigma_g + sigma_x - 2*covmean)

fid_ = FID()

In [ ]:
#Training

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        if i%100==0:
          print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )
        wandb.log({"Disc_loss": d_loss.item(), "Gen_loss": g_loss.item()})
        wandb.log({"Mean_Disc_loss": d_loss.mean().item(), "Mean_Gen_loss": g_loss.mean().item()})

        batches_done = epoch * len(dataloader) + i
        if i==0:
            vutils.save_image(gen_imgs,'/content/images/gen_images_epoch{}_samples{}.png'.format(epoch,i),normalize=True) 
            vutils.save_image(gen_imgs,'/content/drive/MyDrive/DC_GAN/Cifar_GenSamples/generated_images_epoch{}_samples{}.png'.format(epoch,i),normalize=True)
            
        if epoch==n_epochs-1 and i==0:
            fid_val = fid_.compute_fid(real_imgs, gen_imgs)
            print(fid_val)
            with open(('/content/drive/MyDrive/DCGAN_fid_scores.txt'), 'a') as f:
                f.write(f'{epoch},{fid_val}\n')

        if epoch%100==0 and i==0 and epoch>0:
            fid_val = fid_.compute_fid(real_imgs, gen_imgs)
            print(fid_val)
            with open(('/content/drive/MyDrive/DCGAN_fid_scores.txt'), 'a') as f:
                f.write(f'{epoch},{fid_val}\n')
          
wandb.finish()